In [2]:
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added project root to sys.path: {project_root}")

In [3]:
INPUT_LEN = 12  # Number of input time steps
FORECAST_LEN = 6  # Number of forecast time steps
PATH = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"
VARIABLES = [
    'total_precipitation_6hr',
    '2m_temperature', '2m_dewpoint_temperature', 'surface_pressure',
    'mean_sea_level_pressure', '10m_u_component_of_wind', '10m_v_component_of_wind',
    '10m_wind_speed', 'u_component_of_wind', 'v_component_of_wind',
    'total_column_water_vapour', 'integrated_vapor_transport', 'boundary_layer_height',
    'specific_humidity', 'total_cloud_cover',
    'mean_surface_net_short_wave_radiation_flux',
    'mean_surface_latent_heat_flux', 'mean_surface_sensible_heat_flux',
    'snow_depth', 'sea_surface_temperature', 'volumetric_soil_water_layer_1',
    'mean_vertically_integrated_moisture_divergence', 'eddy_kinetic_energy',
    'land_sea_mask'
]
TARGET_VARIABLE = 'total_precipitation_6hr'

In [ ]:
import xarray as xr
import numpy as np
from torch.utils.data import Dataset

class GriddedClimateDataset(Dataset):
    def __init__(self, file_path, input_len=12, forecast_len=1, variables=None,
                 target_variable=None, lat_bounds=(21.0, 5.0), lon_bounds=(97, 106),
                 time_slice=slice("2022-01", "2022-02")):
        self.input_len = input_len
        self.forecast_len = forecast_len
        self.variables = variables
        self.target_variable = target_variable

        # Load dataset (Dask-compatible)
        if file_path.endswith(".nc"):
            self.ds = xr.open_dataset(file_path)
        else:
            self.ds = xr.open_zarr(file_path, consolidated=True, decode_times=False,
                                   storage_options={"token": "anon", "asynchronous": False})
            from cftime import num2date
            units = self.ds.time.attrs.get("units", "hours since 1900-01-01 00:00:0.0")
            cal = self.ds.time.attrs.get("calendar", "standard")
            self.ds["time"] = ("time", num2date(self.ds.time.values, units, calendar=cal))

        # Subset time and region
        self.ds = self.ds.sel(time=time_slice, latitude=slice(*lat_bounds))
        self.ds = self.ds.where((self.ds.longitude >= lon_bounds[0]) | 
                                (self.ds.longitude <= lon_bounds[1]), drop=True)

        # Prepare variable handles (do not load yet)
        self.inputs = {}
        for var in variables:
            da = self.ds[var]
            if "level" in da.dims:
                da = da.mean(dim="level")
            if set(da.dims) == {"latitude", "longitude"}:
                da = da.expand_dims(time=self.ds.time)
            self.inputs[var] = da.chunk({'time': -1})  # Lazy chunking

        self.target = self.ds[target_variable]
        if "level" in self.target.dims:
            self.target = self.target.mean(dim="level")
        self.target = self.target.chunk({'time': -1})

        # Determine sequence count
        self.length = len(self.ds.time) - input_len - forecast_len + 1
        print(f"[Dataset] Initialized with {self.length} samples.")
        print(f"[Dataset] Estimated size: {self.ds.nbytes / 1e6:.2f} MB")

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        t0 = idx
        t1 = idx + self.input_len
        tf = t1 + self.forecast_len

        # Stack variables: [C, T, H, W]
        x_vars = []
        for var in self.variables:
            da = self.inputs[var].isel(time=slice(t0, t1))  # [T, H, W]
            x = da.transpose("time", "latitude", "longitude").values  # triggers lazy load
            x_vars.append(x)

        x_array = np.stack(x_vars, axis=0)  # shape: [C, T, H, W]

        # Target: [forecast_len, H, W]
        y = self.target.isel(time=slice(t1, tf)).transpose("time", "latitude", "longitude").values

        return x_array, y


In [5]:
dataset = GriddedClimateDataset(PATH, input_len=INPUT_LEN, forecast_len=FORECAST_LEN, 
                                  variables=VARIABLES, target_variable=TARGET_VARIABLE,time_slice=slice("1959", "2023"))

c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.1 is exactly one major version older than the runtime version 6.31.1 at api.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\peera\.conda\envs\drought_lstm_base\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Dataset] Initialized with 93527 samples.
[Dataset] Estimated size: 8020243.68 MB


In [7]:
ds = dataset.ds

In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

input_vars = VARIABLES # ← your config['variables']
mins, maxs = [], []
from tqdm import tqdm
for var in tqdm(input_vars, desc="Calculating min/max for scaling"):
    da = ds[var]
    if "level" in da.dims:
        da = da.mean(dim="level")  # Collapse level
    if set(da.dims) == {"latitude", "longitude"}:
        da = da.expand_dims(time=ds.time)  # Broadcast static vars
    mins.append(da.min().compute())
    maxs.append(da.max().compute())

x_data_min = np.array([float(m) for m in mins])
x_data_max = np.array([float(m) for m in maxs])

scaler_x = MinMaxScaler(feature_range=(-1, 1))
scaler_x.min_ = -x_data_min * (2.0 / (x_data_max - x_data_min + 1e-8))
scaler_x.scale_ = 2.0 / (x_data_max - x_data_min + 1e-8)
scaler_x.data_min_ = x_data_min
scaler_x.data_max_ = x_data_max


Calculating min/max for scaling:   0%|          | 0/24 [03:33<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
target_var = TARGET_VARIABLE  # or your config['target_variable']
target = ds[target_var]
if "level" in target.dims:
    target = target.mean(dim="level")
target_min = float(target.min().compute())
target_max = float(target.max().compute())

scaler_y = MinMaxScaler(feature_range=(-1, 1))
scaler_y.min_ = -target_min * (2.0 / (target_max - target_min + 1e-8))
scaler_y.scale_ = 2.0 / (target_max - target_min + 1e-8)
scaler_y.data_min_ = np.array([target_min])
scaler_y.data_max_ = np.array([target_max])
